In [1]:
import plotly.express as px
import numpy as np
import random
from scipy import optimize

import scipy

In [2]:
import mlrose_hiive
import time

In [8]:
restarts = [0, 10, 100, 1000]
restart_curves = []
for res in restarts:
    fit_func = mlrose_hiive.fitness.FlipFlop()
    opt = mlrose_hiive.FlipFlopOpt(length=20, fitness_fn=fit_func)
    start = time.time()
    opt_sol, opt_val, curr_fitness = mlrose_hiive.algorithms.random_hill_climb(
          opt, max_attempts=10, max_iters=100, restarts=1000,
          init_state=None, curve=True, random_state=None,
          state_fitness_callback=None, callback_user_info=None
    )
    end = time.time()
    restart_curves.append((res, opt_sol, opt_val, curr_fitness, end-start))

In [33]:
decays = [('geometric', mlrose_hiive.algorithms.decay.GeomDecay(init_temp=1.0, decay=0.99, min_temp=0.01)),
          ('arithmetic', mlrose_hiive.algorithms.decay.ArithDecay(init_temp=1.0, decay=0.99, min_temp=0.01)),
          ('exponential', mlrose_hiive.algorithms.decay.ExpDecay(init_temp=1.0, exp_const=0.005, min_temp=0.001))]
decay_curves = []
for dec in decays:
    fit_func = mlrose_hiive.fitness.FlipFlop()
    opt = mlrose_hiive.FlipFlopOpt(length=20, fitness_fn=fit_func)
    start = time.time()
    opt_sol, opt_val, curr_fitness = mlrose_hiive.algorithms.simulated_annealing(
        opt, schedule=dec[1],
        max_attempts=10, max_iters=1000, init_state=None, curve=True,
        fevals=False, random_state=None,
        state_fitness_callback=None, callback_user_info=None
    )
    end = time.time()
    print(opt_val)
    decay_curves.append((dec[0], opt_sol, opt_val, curr_fitness, end-start))

19.0
17.0
17.0


In [10]:
pop_percent = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
percent_curves = []

for perc in pop_percent:
    fit_func = mlrose_hiive.fitness.FlipFlop()
    opt = mlrose_hiive.FlipFlopOpt(length=20, fitness_fn=fit_func)
    start = time.time()
    opt_sol, opt_val, curr_fitness = mlrose_hiive.algorithms.genetic_alg(
        opt, pop_size=500, pop_breed_percent=perc, elite_dreg_ratio=0.95,
        minimum_elites=0, minimum_dregs=0, mutation_prob=0.1,
        max_attempts=10, max_iters=100, curve=True, random_state=None
    )
    end = time.time()
    percent_curves.append((perc, opt_sol, opt_val, curr_fitness, end-start))

In [27]:
keep_percent = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
keep_curves = []
for perc in keep_percent:
    print(perc)
    fit_func = mlrose_hiive.fitness.FlipFlop()
    opt = mlrose_hiive.FlipFlopOpt(length=20, fitness_fn=fit_func)
    start = time.time()
    opt_sol, opt_val, curr_fitness = mlrose_hiive.algorithms.mimic(
        opt, pop_size=500, keep_pct=perc, max_attempts=10,
        max_iters=100, curve=True, random_state=None,
        state_fitness_callback=None, callback_user_info=None, noise=0.0
    )
    end = time.time()
    keep_curves.append((perc, opt_sol, opt_val, curr_fitness, end-start))

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0


In [43]:
import pandas as pd
rhc_df = pd.DataFrame(
    [('RHC', curve_tuples[-1], curve_tuples[0], elem[0], elem[1], idx) for curve_tuples in restart_curves for idx, elem in enumerate(curve_tuples[3])],
    columns=['Algorithm', 'Time', 'Restarts', 'Fitness Value', 'Fitness Evaluations', 'Iteration']
)
sa_df = pd.DataFrame(
    [('SA', curve_tuples[-1], curve_tuples[0], elem[0], elem[1], idx) for curve_tuples in decay_curves for idx, elem in enumerate(curve_tuples[3])],
    columns=['Algorithm', 'Time', 'Decay Type', 'Fitness Value', 'Fitness Evaluations', 'Iteration']
)
ga_df = pd.DataFrame(
    [('GA', curve_tuples[-1], curve_tuples[0], elem[0], elem[1], idx) for curve_tuples in percent_curves for idx, elem in enumerate(curve_tuples[3])],
    columns=['Algorithm', 'Time', 'Breeding Percentage', 'Fitness Value', 'Fitness Evaluations', 'Iteration']
)
mimic_df = pd.DataFrame(
    [('MIMIC', curve_tuples[-1], curve_tuples[0], elem[0], elem[1], idx) for curve_tuples in keep_curves for idx, elem in enumerate(curve_tuples[3])],
    columns=['Algorithm', 'Time', 'Keep Percentage', 'Fitness Value', 'Fitness Evaluations', 'Iteration']
)
fig = px.line(rhc_df, x='Iteration', y='Fitness Value', color='Restarts', title='RHC Performance in Flip-Flop with Different Restart Counts')
fig.update_yaxes(range=[5, 20])
fig.update_xaxes(range=[0, 150])
fig.show()

fig = px.line(sa_df, x='Iteration', y='Fitness Value', color='Decay Type', title='SA Performance in Flip-Flop with Different Decay Functions')
fig.update_yaxes(range=[5, 20])
fig.update_xaxes(range=[0, 150])
fig.show()

fig = px.line(ga_df, x='Iteration', y='Fitness Value', color='Breeding Percentage', title='GA Performance in Flip-Flop with Different Breeding Percentages')
fig.update_yaxes(range=[5, 20])
fig.update_xaxes(range=[0, 150])
fig.show()

fig = px.line(mimic_df, x='Iteration', y='Fitness Value', color='Keep Percentage', title='MIMIC Performance in Flip-Flop with Different Keep Percentages')
fig.update_yaxes(range=[5, 20])
fig.update_xaxes(range=[0, 150])
fig.show()